In [14]:
from sys import path
import pandas as pd
import gc

# patching path is dirty trick but acceptable for notebooks
dest_path = '/home/y_ksenia/NGWL/sber-churning/'

if dest_path not in path:
    path.insert(0, dest_path)
    

pd.set_option("display.max_columns", 999)

In [15]:
%load_ext autoreload
%autoreload 2

from joining_tables import get_train, get_shipments, train_test_split

from feature_extraction import FeatureExtractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
train = get_train('./')
train, holdout = train_test_split(train)

In [17]:
fe = FeatureExtractor()

In [20]:
features = fe.exract_all()

In [22]:
train_full = train[['phone_id','prev_month','target']].merge(
    features, left_on=['phone_id','prev_month'], right_on=['phone_id','month']
)

In [23]:
X_train = train_full.drop(['phone_id','prev_month','target'], axis=1).values
y_train = train_full['target'].values

In [33]:
from sklearn.feature_selection import RFECV
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score

In [25]:
from xgboost import XGBClassifier

model = XGBClassifier()

In [26]:
model.fit(X_train, y_train, verbose=True)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [28]:
test_full = holdout[['phone_id','prev_month','target']].merge(
    features, left_on=['phone_id','prev_month'], right_on=['phone_id','month']
)

In [29]:
test_full = test_full[train_full.columns]

In [30]:
X_test = test_full.drop(['phone_id','prev_month','target'], axis=1).values
y_test = test_full['target'].values

In [31]:
preds = model.predict(X_test)

In [34]:
f1_score(y_test, preds)

0.7748558571227674